In [13]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


ERROR: Could not find a version that satisfies the requirement cuda (from versions: none)
ERROR: No matching distribution found for cuda


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [ ]:
print(torch.cuda.is_available())

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0980,  0.1706,  0.3855,  0.2550, -0.1429,  0.2167,  0.2701,  0.4050,
          0.0248, -0.1568,  0.2729,  0.1235, -0.1116,  0.2632,  0.0579,  0.0632,
          0.1141,  0.4719, -0.3110,  0.0692],
        [-0.1724,  0.0109,  0.1749,  0.3035, -0.3157,  0.4358,  0.1495,  0.3039,
          0.1486, -0.2880,  0.3130,  0.2405, -0.1911,  0.1756, -0.2753,  0.4542,
          0.1694,  0.1302, -0.1702, -0.0906],
        [-0.1782,  0.0368,  0.4991,  0.3661,  0.0284, -0.0675,  0.0009,  0.0243,
         -0.0161,  0.0494, -0.0078,  0.2429, -0.3465,  0.2222, -0.0117,  0.0090,
          0.4942,  0.3778, -0.4386, -0.0210]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1706, 0.3855, 0.2550, 0.0000, 0.2167, 0.2701, 0.4050, 0.0248,
         0.0000, 0.2729, 0.1235, 0.0000, 0.2632, 0.0579, 0.0632, 0.1141, 0.4719,
         0.0000, 0.0692],
        [0.0000, 0.0109, 0.1749, 0.3035, 0.0000, 0.4358, 0.1495, 0.3039, 0.1486,
         0.0000, 0.3130, 0.2405, 0.0000, 0.1756, 0.00

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0124,  0.0319, -0.0019,  ...,  0.0073, -0.0317,  0.0132],
        [ 0.0024, -0.0073, -0.0218,  ...,  0.0354,  0.0247,  0.0236]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0053, 0.0160], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 3.1535e-03, -1.9093e-02, -9.0651e-03,  ..., -6.7853e-03,
          3.4465e-02,  7.2487e-05],
        [-1.2922e-02,  1.9334e-02,  2.4821e-02,  ...,  1.4812e-02,
          1.2143e-02, -3.2245e-02]], grad_